In [1]:
import json
import requests
import pickle
import glob

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
with open('creds.txt', 'r') as f:
    name, user, pw, nasapass, token = f.read().split('\n')

In [3]:
def make_box(lat, lon, side_length = 1):
    a = 6378137
    b = 6356752.3142
    e = 0.00669438000426
    
    dx = (0.5 * side_length * 1000) / (a * np.cos(lon) / np.sqrt(1 - e*np.sin(lon)**2))
    dy = (0.5 * side_length * 1000) / (a * (1 - e) / np.sqrt(1 - e * np.sin(lon)**2)**3)
    
    return "W{} N{} E{} S{}".format(lat-dx, lon+dy, lat+dx, lon-dy)

In [4]:
#https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/details?products=VNP14&temporalRanges=2019-10-23..2019-11-06&regions=[BBOX]W-125 N40 E-119.6 S36.5

targets = glob.glob('data/epa/*.csv')

In [5]:
bdate = '2023-05-26'
edate = '2023-06-09'

for target in targets:
    temp = pd.read_csv(target)
    sites = list(set(temp['site_number']))
    
    for site in sites:
        lat = temp['latitude'][temp['site_number'] == site][0]
        long = temp['longitude'][temp['site_number'] == site][0]
        
        bdate = (min(temp['date_local'][temp['site_number'] == site]))
        edate = (max(temp['date_local'][temp['site_number'] == site]))
        
        box = make_box(lat, long)
        
        modis_api = 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/details?products=MOD04_3K&temporalRanges={}..{}&regions=[BBOX]{}'.format(bdate, edate, box)
        response = requests.get(modis_api)
        data = response.json()
        
        print(data)
        
        hdfs = []
        for i in range(len(data['content'])):
            hdfs.append(data['content'][i]['downloadsLink'])

{'content': [{'archiveSets': 61, 'cksum': '2779882968', 'collections': '61', 'dataDay': '2021-001 = 2021-01-01', 'downloadsLink': 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.0615.061.2021268052118.hdf', 'fileId': 6505394329, 'illuminations': 'D', 'md5sum': '4b582b9e5f565e515ba82bcbf5ef0181', 'mtime': 1632547390, 'name': 'MOD04_3K.A2021001.0615.061.2021268052118.hdf', 'products': 'MOD04_3K', 'resourceType': 'File', 'self': '/api/v2/content/details/MOD04_3K.A2021001.0615.061.2021268052118.hdf', 'size': 3773561}, {'archiveSets': 61, 'cksum': '1953030569', 'collections': '61', 'dataDay': '2021-001 = 2021-01-01', 'downloadsLink': 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.1930.061.2021268052428.hdf', 'fileId': 6505398012, 'illuminations': 'D', 'md5sum': 'd8493f36f8a375304f55c7bcdb48c3da', 'mtime': 1632547566, 'name': 'MOD04_3K.A2021001.1930.061.2021268052428.hdf', 'products': 'MOD04_3K', 'resourceType': 'File', 'se

In [6]:
hdfs

['https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.0615.061.2021268052118.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.1930.061.2021268052428.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.0935.061.2021268052417.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.1750.061.2021268052600.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.0755.061.2021268052005.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.1255.061.2021268052226.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.0620.061.2021268052102.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.1935.061.2021268052631.hdf',
 'https://ladsweb.modaps.eosdis.nasa.gov/api/v2/content/archives/MOD04_3K.A2021001.1610.061.202126805272

In [7]:
import requests as r
import getpass, pprint, time, os, cgi, json

inDir = '.'           # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                      # Change to working directory
api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

In [8]:
token_response = r.post('{}login'.format(api), auth=(user, nasapass)).json() # Insert API URL, call login service, provide credentials & return json
#del user, nasapass                                                           # Remove user and password information
token_response                                                               # Print response

{'token_type': 'Bearer',
 'token': 'LG3lcEqpTtCA7jA-DHB36Rm_Rlswx52bw1Svis1PlS6A6nJ58ACJfXoqt5cOY_CSoY6c6df7RVYw9AVcLKROtw',
 'expiration': '2023-06-26T02:04:42Z'}

In [9]:
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
print('AρρEEARS currently supports {} products.'.format(len(product_response)))  # Print no. products available in AppEEARS

AρρEEARS currently supports 169 products.


In [10]:
#product_response

In [11]:
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version
print(products.keys())

dict_keys(['GPW_DataQualityInd.411', 'GPW_UN_Adj_PopCount.411', 'GPW_UN_Adj_PopDensity.411', 'GPW_Basic_Demog_Char.411', 'MCD12Q1.006', 'MCD12Q2.006', 'MCD12Q1.061', 'MCD12Q2.061', 'MCD15A2H.006', 'MCD15A2H.061', 'MCD15A3H.006', 'MCD15A3H.061', 'MCD43A1.006', 'MCD43A1.061', 'MCD43A2.061', 'MCD43A3.006', 'MCD43A3.061', 'MCD43A4.006', 'MCD43A4.061', 'MCD64A1.006', 'MCD64A1.061', 'MOD09A1.006', 'MOD09A1.061', 'MOD09GA.006', 'MOD09GA.061', 'MOD09GQ.006', 'MOD09GQ.061', 'MOD09Q1.006', 'MOD09Q1.061', 'MOD10A1.006', 'MOD10A1.061', 'MOD10A2.006', 'MOD10A2.061', 'MOD11A1.006', 'MOD11A1.061', 'MOD11A2.006', 'MOD11A2.061', 'MOD13A1.006', 'MOD13A1.061', 'MOD13A2.006', 'MOD13A2.061', 'MOD13A3.006', 'MOD13A3.061', 'MOD13Q1.006', 'MOD13Q1.061', 'MOD14A2.006', 'MOD14A2.061', 'MOD15A2H.006', 'MOD15A2H.061', 'MOD16A2.006', 'MOD16A2.061', 'MOD16A2GF.006', 'MOD16A2GF.061', 'MOD16A3GF.006', 'MOD16A3GF.061', 'MOD17A2H.006', 'MOD17A2H.061', 'MOD17A2HGF.006', 'MOD17A2HGF.061', 'MOD17A3HGF.006', 'MOD17A3HGF.06

In [12]:
products['MOD09GA.061']

{'Product': 'MOD09GA',
 'Platform': 'Terra MODIS',
 'Description': 'Surface Reflectance Bands 1-7',
 'RasterType': 'Tile',
 'Resolution': '500/1000m',
 'TemporalGranularity': 'Daily',
 'Version': '061',
 'Available': True,
 'DocLink': 'https://doi.org/10.5067/MODIS/MOD09GA.061',
 'Source': 'LP DAAC',
 'TemporalExtentStart': '2000-02-24',
 'TemporalExtentEnd': 'Present',
 'Deleted': False,
 'DOI': '10.5067/MODIS/MOD09GA.061',
 'Info': {},
 'ProductAndVersion': 'MOD09GA.061'}

In [22]:
import h5py

filename = './MOD04_3K.A2021001.1115.061.2021268052353.h5'

f = h5py.File(filename)

for key in f.keys():
    print(key) #Names of the root level object names in HDF5 file - can be groups or datasets.
    print(type(f[key])) # get the object type: usually group or dataset


Cell_Across_Swath:mod04
<class 'h5py._hl.dataset.Dataset'>
Cell_Along_Swath:mod04
<class 'h5py._hl.dataset.Dataset'>
MODIS_Band_Land:mod04
<class 'h5py._hl.dataset.Dataset'>
MODIS_Band_Ocean:mod04
<class 'h5py._hl.dataset.Dataset'>
QA_Byte_Land:mod04
<class 'h5py._hl.dataset.Dataset'>
QA_Byte_Ocean:mod04
<class 'h5py._hl.dataset.Dataset'>
Solution_1_Land:mod04
<class 'h5py._hl.dataset.Dataset'>
Solution_2_Land:mod04
<class 'h5py._hl.dataset.Dataset'>
Solution_3_Land:mod04
<class 'h5py._hl.dataset.Dataset'>
Solution_Index:mod04
<class 'h5py._hl.dataset.Dataset'>
Solution_Ocean:mod04
<class 'h5py._hl.dataset.Dataset'>
mod04
<class 'h5py._hl.group.Group'>


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [14]:
#Get the HDF5 group; key needs to be a group name from above
group = f[key]

#Checkout what keys are inside that group.
for key in group.keys():
    print(key)

Data Fields
Geolocation Fields
Swath Attributes


In [15]:
# This assumes group[some_key_inside_the_group] is a dataset, 
# and returns a np.array:
data1 = group['Geolocation Fields']
#Do whatever you want with data

data1['Latitude'][:]

array([[-68.86022 , -68.980736, -69.09829 , ..., -80.51433 , -80.48345 ,
        -80.4501  ],
       [-68.88531 , -69.00572 , -69.12317 , ..., -80.567345, -80.53674 ,
        -80.5037  ],
       [-68.910164, -69.03048 , -69.147835, ..., -80.62031 , -80.59001 ,
        -80.55727 ],
       ...,
       [-69.14676 , -69.268394, -69.38705 , ..., -81.08495 , -81.05278 ,
        -81.01788 ],
       [-69.12348 , -69.2452  , -69.36394 , ..., -81.032   , -80.99954 ,
        -80.96448 ],
       [-69.09999 , -69.221794, -69.34061 , ..., -80.97901 , -80.94621 ,
        -80.910995]], dtype=float32)

In [18]:
# This assumes group[some_key_inside_the_group] is a dataset, 
# and returns a np.array:
data2 = group['Data Fields']
#Do whatever you want with data

for key in data2.keys():
    print(key)
    
print(data2['Optical_Depth_Large_Best_Ocean'][0][0])

Aerosol_Cloud_Fraction_Land
Aerosol_Cloud_Fraction_Ocean
Aerosol_Type_Land
Angstrom_Exponent_1_Ocean
Angstrom_Exponent_2_Ocean
Asymmetry_Factor_Average_Ocean
Asymmetry_Factor_Best_Ocean
Backscattering_Ratio_Average_Ocean
Backscattering_Ratio_Best_Ocean
BowTie_Flag
Corrected_Optical_Depth_Land
Corrected_Optical_Depth_Land_wav2p1
Effective_Optical_Depth_Average_Ocean
Effective_Optical_Depth_Best_Ocean
Effective_Radius_Ocean
Fitting_Error_Land
Glint_Angle
Image_Optical_Depth_Land_And_Ocean
Land_Ocean_Quality_Flag
Land_sea_Flag
Least_Squares_Error_Ocean
MODIS_Band_Land
MODIS_Band_Land_t
MODIS_Band_Ocean
MODIS_Band_Ocean_t
Mass_Concentration_Land
Mass_Concentration_Ocean
Mean_Reflectance_Land
Mean_Reflectance_Ocean
Number_Pixels_Used_Land
Number_Pixels_Used_Ocean
Optical_Depth_Land_And_Ocean
Optical_Depth_Large_Average_Ocean
Optical_Depth_Large_Best_Ocean
Optical_Depth_Ratio_Small_Land
Optical_Depth_Ratio_Small_Ocean_0.55micron
Optical_Depth_Small_Average_Ocean
Optical_Depth_Small_Best_Ocea

In [17]:
data3 = group['Swath Attributes']

for key in data3.keys():
    print(key)

_FV_Aerosol_Cloud_Fraction_Land
_FV_Aerosol_Cloud_Fraction_Land_t
_FV_Aerosol_Cloud_Fraction_Ocean
_FV_Aerosol_Cloud_Fraction_Ocean_t
_FV_Aerosol_Type_Land
_FV_Aerosol_Type_Land_t
_FV_Angstrom_Exponent_1_Ocean
_FV_Angstrom_Exponent_1_Ocean_t
_FV_Angstrom_Exponent_2_Ocean
_FV_Angstrom_Exponent_2_Ocean_t
_FV_Asymmetry_Factor_Average_Ocean
_FV_Asymmetry_Factor_Average_Ocean_t
_FV_Asymmetry_Factor_Best_Ocean
_FV_Asymmetry_Factor_Best_Ocean_t
_FV_Backscattering_Ratio_Average_Ocean
_FV_Backscattering_Ratio_Average_Ocean_t
_FV_Backscattering_Ratio_Best_Ocean
_FV_Backscattering_Ratio_Best_Ocean_t
_FV_BowTie_Flag
_FV_BowTie_Flag_t
_FV_Corrected_Optical_Depth_Land
_FV_Corrected_Optical_Depth_Land_t
_FV_Corrected_Optical_Depth_Land_wav2p1
_FV_Corrected_Optical_Depth_Land_wav2p1_t
_FV_Effective_Optical_Depth_Average_Ocean
_FV_Effective_Optical_Depth_Average_Ocean_t
_FV_Effective_Optical_Depth_Best_Ocean
_FV_Effective_Optical_Depth_Best_Ocean_t
_FV_Effective_Radius_Ocean
_FV_Effective_Radius_Ocean_